# Домашнее задание №2

**

## Часть 1. ML workflow (**всего 5 баллов**)

In [225]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

from sklearn.linear_model import SGDRegressor

### Загрузим данные для работы. 

In [226]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [227]:
# Y - последний столбец, X - все остальные
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [228]:
# Используем функцию train_test_split для этой цели
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [229]:
# Используем функцию LinearRegression(), чтобы сделать предсказание

lr1 = LinearRegression().fit(X_train, y_train)

y_pred1_train = lr1.predict(X_train)
y_pred1_test = lr1.predict(X_test)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.

In [230]:
# Посчитаем MSE для Train и Test выборок
mse_train = mean_squared_error(y_train, y_pred1_train)
mse_test = mean_squared_error(y_test, y_pred1_test)

# Посчитаем R2 для Train и Test выборо
r2_train = r2_score(y_train, y_pred1_train)
r2_test = r2_score(y_test, y_pred1_test)

# Выведем полученные значения
print('mse_train:', mse_train)
print('mse_test:', mse_test)

print('r2_train:', r2_train)
print('r2_test:', r2_test)

mse_train: 0.4189162445851497
mse_test: 0.41987748164199784
r2_train: 0.3748607620409803
r2_test: 0.30961311573808714


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [231]:
# Используем r2 как score, который подсчитывается на кросс валидации
from sklearn.model_selection import cross_validate

num_splits=5

cv_res = cross_validate(lr1, X, y, scoring='r2', cv=num_splits)

print('mean_r2_cv:', np.mean(cv_res['test_score']))

mean_r2_cv: 0.2900416288421967


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [232]:
# Используем кроссвалидацию, чтобы найти оптимальный гипермпараметр alpha регуляризации
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso = Lasso()
num_splits = 5
alphas = np.linspace(0.1,1.1,11)
params = {'alpha':alphas}

cv = GridSearchCV(lasso, params, scoring='r2', cv=num_splits)
cv.fit(X_train, y_train)

# Это и есть наш оптимальный гиперпараметр
print(cv.best_params_)

{'alpha': 0.1}


### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [233]:
# Используем оптимальный альфа из прошлого пункта, чтобы посчитать лучший r2 на тренировочных данных, который мы можем получить в LASSO модели
print('best_estimator:', cv.best_estimator_)
print('best_score:', cv.best_score_)

best_estimator: Lasso(alpha=0.1)
best_score: 0.2533626619326174


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [234]:
# Используя best estimator вывели r2 на тесте
best_est = cv.best_estimator_
y_pred = best_est.predict(X_test)

r2_test = r2_score(y_test, y_pred)
print('r2_test:', r2_test)

r2_test: 0.2106617839974564


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [235]:
# Добавим полиномиальные признаки в модель, используем pipeline для этого
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

#your code here
pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('model_', LinearRegression())
    ])

In [236]:
cv_res = cross_validate(pipe, X, y, scoring='r2', cv=5)
print('r2 pipe:', np.mean(cv_res['test_score']))

r2 pipe: 0.23009616556632376


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [237]:
# Используя pipeline обучим регрессию и сделаем прогноз
pipe.fit(X_train, y_train)
y_pipe_pred_train = pipe.predict(X_train)
y_pipe_pred_test = pipe.predict(X_test)

In [238]:
mse_train = mean_squared_error(y_train, y_pipe_pred_train)
mse_test = mean_squared_error(y_test, y_pipe_pred_test)

r2_train = r2_score(y_train, y_pipe_pred_train)
r2_test = r2_score(y_test, y_pipe_pred_test)

print('mse_train:', mse_train)
print('mse_test:', mse_test)

print('r2_train:', r2_train)
print('r2_test:', r2_test)

mse_train: 0.3622378456375558
mse_test: 0.4412735881742095
r2_train: 0.459440655002458
r2_test: 0.2744323976239771


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

### Выводы:
1. MSE ~ 0.45, R^2 ~ 0.32 на тестовых данных не кажется мне хорошим качеством. При этом мы видим, что MSE на train ниже, а R2 на train выше, что указывает на то, что модель дает лучшие результаты на тренировочных данных. Вероятно, здесь мы наблюдаем переобучение
2. Если мой код верный, то R^2 только уменьшился после введения L1-регуляризация, поэтому нет, качество модели не удалось улучшить
3. Добавление полиномов второй степени только ухудшило модель - переобучение стало еще более явным, потому что модель лучше "подогналась" под тренировочные данные

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

## Не получилось
Я пробовал Elastic net и Random Forest с кроссвалидацией для поиска оптимальных гиперпараметров, но это не помогло улучшить качество

## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [239]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [240]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [241]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


#### Способ 2

In [242]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [243]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold

# Давайте создадим еще один столбец и заполним его нулями
all_data['K_fold_scheme'] = np.zeros
kf = KFold(n_splits=5, shuffle=False)

# Проитерируемся по фолдам и подсчитаем средние значения таргета по item_id, заполним ими новый столбец
for train_index, validation_index in kf.split(all_data):
    
    all_data_train, all_data_validation = all_data.iloc[train_index], all_data.iloc[validation_index]
    item_id_target_mean = all_data_train.groupby('item_id').target.mean()
    all_data.loc[all_data.index[validation_index], 'K_fold_scheme'] = all_data_validation['item_id'].map(item_id_target_mean)

# Заполним NaN values 0.3343 (из условия - было в прошлых пунктах способ 1, способ 2)
all_data['K_fold_scheme'] = all_data['K_fold_scheme'].fillna(0.3343)

encoded_feature = all_data['K_fold_scheme'].values
    
# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4164590712798811


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [244]:
# YOUR CODE GOES HERE

# Вычислили суммарный таргет по всем объектам
# Вычислим количество объектов
count_LOO = all_data['item_id'].map(all_data.groupby('item_id').target.count())
sum_LOO = all_data['item_id'].map(all_data.groupby('item_id').target.sum())

# Делить надо именно на количество объектов минус 1
all_data['LOO'] = ((sum_LOO - all_data['target']))/(count_LOO-1)
all_data['LOO'] = all_data['LOO'].fillna(0.3343)

encoded_feature = all_data['LOO'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4803848311293002


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [245]:
# YOUR CODE GOES HERE
alpha = 100
globalmean = 0.3343

# Как указано в условии нам нужно именно таким образом посчитать nrows
nrows = all_data.groupby('item_id').size()
mean = all_data.groupby('item_id')['target'].agg('mean')

# Используем формулу из условия
score = ((mean * nrows)  + globalmean * alpha) / (nrows + alpha)

# Добавим столбец со сглаживанием
all_data['smoothing'] = all_data['item_id']
all_data['smoothing'] = all_data['smoothing'].map(score)

encoded_feature = all_data['smoothing'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4818198797097264


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [246]:
# YOUR CODE GOES HERE
# Используем функции cumsum и cumcoun для предварительных расчетов
cumcount = all_data.groupby('item_id').cumcount()
cumsum = all_data.groupby('item_id').target.cumsum() - all_data['target']

# Посчитаем нужные нам значения
exp_mean = cumsum/cumcount

# Вставим полученные значения в новый столбец датасета и заполним пропуски
all_data['exp_mean'] = exp_mean
all_data['exp_mean'] = all_data['exp_mean'].fillna(0.3343) 

encoded_feature = all_data['exp_mean'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.5025245211081697


Ожидаемый ответ 0.5025